In [1]:
!pip install kiwipiepy rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.17.0-py3-none-any.whl size=34845033 sha256=93bbbc86a988b2ca5f80ace5ba52f202593ae17f4306c71d2cba110269f99061
  Stored in directory: /root/.cache/pip/wheels/90/bb/ae/15e1cf26d17183040bac268fbf8d1ca1f45f7f4daa0fb76b91
Successfully built kiwipiepy-model


In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_qna.csv to sample_qna.csv
Saving sampled_ntis_chat.csv to sampled_ntis_chat.csv


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from kiwipiepy import Kiwi
from rank_bm25 import BM25Okapi
from transformers import pipeline, AutoModelForCausalLM
import torch

In [4]:
kiwi = Kiwi()

In [5]:
# 한국어 토크나이저 함수 정의
def tokenizer(text):
    """
    텍스트를 형태소 단위로 토큰화합니다.

    text: str - 입력 텍스트
    반환값: list - 토큰 리스트
    """
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens]

In [6]:
def chatbot(query):

    if '#LLM' in query:
        output = llm(query.split('#LLM')[1])
        return output

    intent = run_intent_classifier(query, model, vectorizer, le_name_mapping)
    # 'IntroduceServices', 'Search', 'QnA', 'Statistics'
    if intent == 'IntroduceServices':
        pass
    elif intent == 'Search':
        pass
    elif intent == 'QnA':
        output = qna(query, bm25)
    elif intent == 'Statistics':
        pass

    return output


In [7]:
def train_intent_classifier():
    data = pd.read_csv('sampled_ntis_chat.csv')
    X_train, X_test, y_train, y_test = train_test_split(data['question'], data['label'], test_size=0.2, random_state=777, stratify=data['label'])
    le = LabelEncoder()
    le.fit(data['label'])

    le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))

    y_train = le.transform(y_train)
    y_test = le.transform(y_test)

    vectorizer = TfidfVectorizer(max_features=1000)  # max_features는 조정 가능
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    regr = RandomForestClassifier(random_state=0)
    regr.fit(X_train_tfidf, y_train)
    return regr, vectorizer, le_name_mapping

In [18]:
def run_intent_classifier(query, model, vectorizer, le_name_mapping):
  X_test_tfidf = vectorizer.transform([query])
  y_pred = model.predict(X_test_tfidf)
  return le_name_mapping[y_pred[0]]


In [8]:
model, vectorizer, le_name_mapping = train_intent_classifier()

In [9]:
def tokenizer(oneexample):
    tokens = kiwi.tokenize(oneexample)
    tokenized_user = []
    for token in tokens:
        tokenized_user.append(token.form)
    return (tokenized_user)

def make_index(examples):
    tokenized_user = []
    for idx in examples.index:
        sentence = examples.loc[idx,"문의내용"]
        tokens = kiwi.tokenize(sentence)
        tokenized_example = []
        for token in tokens:
            tokenized_example.append(token.form)
        tokenized_user.append(tokenized_example)

    bm25 = BM25Okapi(tokenized_user)
    return(bm25)

def preprocess_function(examples):
    user = [examples.loc[idx,"문의내용"] for idx in examples.index ]
    responses = [examples.loc[idx,"답변"] for idx in examples.index]

    dict_response={}
    for i in range(len(user)):
        dict_response[user[i]] = responses[i]

    return dict_response

qna = pd.read_csv('sample_qna.csv', encoding='utf-8', index_col=False)
qna = qna.loc[:22,:]
bm25 = make_index(qna)

dict_response = preprocess_function(qna)
user_data = [qna.loc[idx,"문의내용"] for idx in qna.index ]



In [10]:
LLM_MODEL = "42dot/42dot_LLM-SFT-1.3B"

llm_model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.float16,
)
pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=LLM_MODEL,
    device=0,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.76G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/529k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
def llm(query):
  ans = pipe(
              f"### AI 챗봇:\n{query}\n\n### 사용자:",
              do_sample=True,
              max_new_tokens=512,
              temperature=0.7,
              top_p=0.5,
              return_full_text=False,
            )

  msg = ans[0]["generated_text"]
  if len(msg.split('### AI 챗봇:')[0]) > 0:
      output = msg.split('### AI 챗봇:')[1]
      if len(output.split('###')) > 1:
        output = output.split('###')[0]
  else:
      output = '답변을 드리지 못하여 죄송합니다.'
  return output.strip()

In [12]:
def qna(query, bm25):
    tokenized_query= tokenizer(query)
    matched_query = ' '.join(bm25.get_top_n(tokenized_query, user_data, n=1))
    match_score = max(bm25.get_scores(tokenized_query))
    response = dict_response[matched_query]
    return response

In [23]:
print(chatbot("비밀번호 까먹었다."))
print(chatbot("#LLM 과학기술 발전이 중요한 이유 말해줘."))

확인이 필요한 사항이니 NTIS센터 콜센터(042-869-1115)로 연락을 주시면 처리해드리겠습니다.
과학기술 발전은 우리의 삶을 크게 변화시킬 수 있는 잠재력을 가지고 있습니다. 예를 들어, 인공지능 기술은 의료, 교통, 교육 등 다양한 분야에서 혁신을 가져올 수 있습니다. 인공지능은 의료 분야에서 질병 진단과 치료에 도움을 줄 수 있고, 교통 분야에서는 자율주행 자동차와 같은 혁신적인 기술을 개발할 수 있습니다. 또한, 교육 분야에서는 개인 맞춤형 학습 경험을 제공하고, 학습 효과를 극대화할 수 있습니다. 이러한 기술들은 우리의 삶을 편리하고 효율적으로 만들어 줄 수 있으며, 경제적인 발전과 사회적 발전을 촉진할 수 있습니다.
